 # <p style="text-align: center;">Prevendo a Propensidade de Compra do Consumidor</p> 

Neste exemplo, mostraremos como prever a propensão dos clientes potenciais. Usaremos dados de cliques na web sobre os links clicados pelo usuário enquanto ele navega para prever sua propensão a comprar o produto. Usando essa propensão, decidiremos se queremos criar um ponto de contato, neste caso enviar uma propaganda nas mídias sociais com 10% de desconto sobre o produto.

## Gravando e Acessando os Dados
Carregaremos o arquivo de dados para este exemplo e verificaremos as estatísticas e colunas resumidas desse arquivo.

In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.model_selection  import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

prospect_data = pd.read_csv("C:/Users/User/Desktop/target_app.csv")

prospect_data.dtypes

SESSION_ID          int64
Click_Image         int64
Read_Reviews        int64
Category_View       int64
Read_Details        int64
Video_View          int64
Add_to_List         int64
Compare_Products    int64
View_Similar        int64
Save_for_Later      int64
Personalized        int64
BUY                 int64
dtype: object

Os dados contêm informações sobre os diversos links do site que são clicados pelo usuário durante sua navegação. Esses são dados anteriores que serão usados ​​para construir o modelo.

- Session ID : um identificador exclusivo para a sessão de navegação na web
- Buy: se o cliente potencial acabou comprando o produto
- Outras colunas: um indicador de 0 ou 1 para mostrar se o cliente potencial visitou aquela página específica ou realizou a atividade mencionada.


In [3]:
# Inspecionando os Dados Disponíveis
prospect_data.head()

,SESSION_ID,Click_Image,Read_Reviews,Category_View,Read_Details,Video_View,Add_to_List,Compare_Products,View_Similar,Save_for_Later,Personalized,BUY
0,27873,0,0,1,0,1,0,0,0,1,0,0
1,27874,0,1,1,0,0,0,0,0,0,1,0
2,27875,1,0,1,1,1,0,0,0,1,0,0
3,27876,1,0,0,0,1,1,1,0,0,0,0
4,27877,1,1,1,0,1,0,1,0,0,0,0


In [4]:
#Fazendo uma análise estatística resumida dos dados
prospect_data.describe()

,SESSION_ID,Click_Image,Read_Reviews,Category_View,Read_Details,Video_View,Add_to_List,Compare_Products,View_Similar,Save_for_Later,Personalized,BUY
count,500.000000,500.000000,500.0000,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,28122.500000,0.510000,0.5200,0.440000,0.4800,0.528000,0.500000,0.580000,0.468000,0.532000,0.550000,0.370000
std,144.481833,0.500401,0.5001,0.496884,0.5001,0.499715,0.500501,0.494053,0.499475,0.499475,0.497992,0.483288
min,27873.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27997.750000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28122.500000,1.000000,1.0000,0.000000,0.0000,1.000000,0.500000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,28247.250000,1.000000,1.0000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,28372.000000,1.000000,1.0000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Realizando análise de correlação

In [5]:
prospect_data.corr()['BUY']

SESSION_ID          0.026677
Click_Image         0.046819
Read_Reviews        0.404628
Category_View      -0.095136
Read_Details        0.009950
Video_View         -0.022239
Add_to_List        -0.103562
Compare_Products    0.190522
View_Similar       -0.096137
Save_for_Later      0.179156
Personalized        0.110328
BUY                 1.000000
Name: BUY, dtype: float64

Observando as correlações acima podemos ver que alguns recursos como Read_Reviews, Compare_Products, Add_to_List, Save_for_Later e Personalized possuem correlação média com a variável alvo. Reduziremos nosso conjunto de recursos a essa lista de variáveis.

In [8]:
#Eliminar colunas com baixa correlação
predictors = prospect_data[['Read_Reviews','Compare_Products','Add_to_List','Save_for_Later','Personalized','View_Similar']]
targets = prospect_data.BUY


##  Divisão de treinamento e teste

Agora dividimos o modelo em dados de treinamento e teste na proporção de 70:30

In [9]:
pred_train, pred_test, tar_train, tar_test  =   train_test_split(predictors, targets, test_size=.3)

print( "Predictor - Training : ", pred_train.shape, "Predictor - Testing : ", pred_test.shape )


Predictor - Training :  (350, 6) Predictor - Testing :  (150, 6)


## Construir modelo e verificar a precisão

In [10]:
from sklearn.naive_bayes import GaussianNB

classifier=GaussianNB()
classifier=classifier.fit(pred_train,tar_train)

predictions=classifier.predict(pred_test)

#Analyze accuracy of predictions
sklearn.metrics.confusion_matrix(tar_test,predictions)


array([[80, 21],
       [25, 24]], dtype=int64)

In [11]:
sklearn.metrics.accuracy_score(tar_test, predictions)

0.6933333333333334

Em vez de fazer uma previsão Sim/Não, podemos fazer um cálculo de probabilidade para mostrar a probabilidade do cliente potencial comprar o produto

In [12]:
pred_prob=classifier.predict_proba(pred_test)
pred_prob[0,1]

0.06503038919293792

A probabilidade acima pode ser lida como X% de chance de o cliente potencial comprar o produto.

## Previsões em tempo real

Agora que o modelo foi construído, vamos usá-lo para previsões em tempo real. Assim, quando o cliente começa a visitar as páginas uma por uma, coletamos essa lista e a usamos para calcular a probabilidade. Fazemos isso para cada novo clique que chega.

Então vamos começar. O cliente em potencial acabou de acessar seu site. Não há cliques significativos. Vamos calcular a probabilidade. A matriz de valores passada possui os valores para 'Read_Reviews','Compare_Products','Add_to_List','Save_for_Later','Personalized'. Então a matriz é toda zeros para começar

In [13]:
browsing_data = np.array([0,0,0,0,0,0]).reshape(1, -1)
print("New User: propensity :",classifier.predict_proba(browsing_data)[:,1] )


New User: propensity : [0.04645303]


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Portanto, a probabilidade inicial é de 4%. Agora, suponha que o cliente faça uma comparação de produtos semelhantes. A matriz muda para incluir 1 para essa função. A nova probabilidade será

In [14]:
browsing_data = np.array([0,0,1,0,0,0]).reshape(1, -1)
print("After Add_to_List: propensity :",classifier.predict_proba(browsing_data)[:,1] )


After Add_to_List: propensity : [0.03712185]


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Ele sobe. Em seguida, ele verifica as avaliações.

In [15]:
browsing_data = np.array([0,1,0,0,0,0]).reshape(1, -1)
print("After Compare_Products e Add_to_List: propensity :",classifier.predict_proba(browsing_data)[:,1] )


After Compare_Products e Add_to_List: propensity : [0.1272649]


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Ele dispara até 50+%. Você pode definir um limite para quando deseja oferecer bate-papo. Você pode continuar verificando essa probabilidade em relação a esse limite para ver se deseja abrir uma janela de bate-papo.

Este exemplo mostra como você pode usar a análise preditiva em tempo real para decidir se um cliente potencial tem alta propensão para conversão e oferecer a ele um bate-papo com um representante/agente de vendas.